# TASK TARGET

任務1：載入英雄聯盟資料集並對其進行預處理以進行訓練（10分）

導入必要的函式庫（pandas、train_test_split、StandardScaler、torch）。

使用 pd.read_csv 載入資料集。

將資料拆分為特徵 (X) 和目標 (y)。

使用train_test_split分割資料集。

使用 StandardScaler 標準化功能。

將資料轉換為 PyTorch 張量。

任務 2：使用 PyTorch 實現邏輯迴歸模型（5 分）

導入 torch.nn 和 torch.optim。

定義一個繼承自nn.Module的類別LogisticRegressionModel。

實作 __init__ 和轉發方法。

初始化模型、損失函數 (nn.BCELoss) 和最佳化器 (optim.SGD)。

任務 3：在資料集上訓練邏輯迴歸模型。 （5分）

實現指定次數的訓練循環。

進行預測並計算損失。

執行反向傳播並更新模型參數。

評估訓練和測試集上的模型和列印準確性。

任務 4：實施最佳化技術並評估模型的效能。 （5分）

在優化器中實作 L2 正規化（weight_decay 參數）。

使用相同的訓練循環重新訓練模型。

評估訓練和測試集上的最佳化模型。

任務 5：可視化模型的表現並解釋結果。 （8分）

導入必要的函式庫（matplotlib.pyplot、confusion_matrix、classification_report、roc_curve、auc）。

產生並繪製混淆矩陣。

列印分類報告。

繪製 ROC 曲線並計算 AUC。

任務 6：儲存並載入經過訓練的模型。 （8分）

使用 torch.save 儲存模型的狀態字典。

使用 torch.load 將狀態字典載入到新的模型實例中。

將載入的模型設定為評估模式。

評估加載的模型並確保一致的性能。

任務 7：執行超參數調整以找到最佳學習率。 （8分）

定義要測試的學習率清單。

為每個學習率重新初始化模型和最佳化器。

針對每個學習率訓練和評估模型。

列印最佳學習率和相應的測試準確率。

任務 8：評估特徵重要性以了解每個特徵對預測的影響。 （6分）

從線性層中提取權重。

建立一個包含特徵名稱及其對應重要性的 DataFrame。

依重要性對 DataFrame 進行排序。

使用長條圖繪製特徵重要性。

# https://github.com/kaushikilango/league-game-result-predictor

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.metrics  import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import f1_score,recall_score,precision_score,accuracy_score,roc_auc_score,confusion_matrix,classification_report,roc_curve,auc
from torch.utils.data import Dataset,DataLoader, TensorDataset
import torch.optim as optim

# step 1
data = pd.read_csv('/content/league_of_legends_data_large.csv') # read file
data = data.dropna()
X = data.drop('win', axis=1)
y = data['win']
X = StandardScaler().fit_transform(X)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


# step 2 define model
class LogisticRegressionModel(nn.Module):
  def __init__(self,input_dim):
    super(LogisticRegressionModel,self).__init__()
    self.linear = nn.Linear(input_dim,1) # because the y is only the two classes

  def forward(self,x):
    yhat = torch.sigmoid(self.linear(x)) # two classes use sigmoid
    return yhat

model = LogisticRegressionModel(X_train.shape[1])
optimizer = optim.SGD(model.parameters())
criterion = nn.BCELoss() # binary cross entropy loss

# step 3 model training
def train_model(epochs=1000):
  for epoch in range(epochs):
    for x,y in train_loader:
      outputs = model(x)
      y = y.unsqueeze(1)
      loss = criterion(outputs,y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    if epoch % 100 == 0:
      print(f"the current epoch is {epoch} and the loss value is {loss}")
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for X_batch, y_batch in test_loader:
        test_outputs = model(X_batch)
        y_batch = y_batch.unsqueeze(1)
        loss = criterion(test_outputs, y_batch)
        test_loss += loss.item()

train_model(1000)

def evaluate_acc(model,data,threshold = 0.5):
  model.eval()
  pred_list =[]
  target_list = []
  with torch.no_grad():
    for X,y in data:
      output = model(X)
      preds = (output >= threshold).float()
      pred_list.append(preds.squeeze().numpy())
      target_list.append(y.squeeze().numpy())
  accuracy = accuracy_score(pred_list, target_list)
  return accuracy , pred_list , target_list

train_accuracy,train_pred,train_target = evaluate_acc(model, train_loader)
test_accuracy,test_pred,test_target = evaluate_acc(model, test_loader)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Testing Accuracy: {test_accuracy:.2f}")

# step 4 Model Optimization and Evaluation
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
train_model(1000)
train_accuracy,train_pred,train_target = evaluate_acc(model, train_loader)
test_accuracy,test_pred,test_target = evaluate_acc(model, test_loader)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Testing Accuracy: {test_accuracy:.2f}")

# step 5 Visualization and Interpretation
def plot_confusion_matrix(y_true,y_pred):
  cm = confusion_matrix(y_true, y_pred)
  cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1])
  cm_display.plot()
  plt.show()

def plot_roc_curve(y_true,y_pred):
  fpr,tpr,threshold= roc_curve(y_true, y_pred)
  roc_auc =auc(fpr, tpr)
  plt.xlabel('FP rate')
  plt.ylabel('TP rate')
  plt.title('roc curve')
  plt.plot(fpr,tpr,color='b',linewidth=0.8)
  plt.plot([0,1],[0,1],'r--')
  plt.show()

def predict_score(y_true,y_pred):
  precision_score_ = precision_score(y_true, y_pred, average='binary')
  recall_score_ = recall_score(y_true, y_pred, average='binary')
  f1_score_ = f1_score(y_true, y_pred, average='binary')
  return precision_score_,recall_score_,f1_score_

plot_confusion_matrix(test_target,test_pred)
plot_roc_curve(test_target,test_pred)

precision_score,recall_score,f1_score = predict_score(test_target,test_pred)

#



ModuleNotFoundError: No module named 'torch'